# Import Libraries

In [36]:
params = {}

In [37]:
import time
import json
import sys

from data_task_helpers import something

something()

from api_data_task_executioner.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task, find_json_arg  # noqa: E402


something
['/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/.venv/lib/python3.11/site-packages', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/duft', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/duft-server']


In [38]:
   
environment = initialise_data_task("Jupyter Sample Data Task", params=params)
params["name"] = params.get("customname", params.get("name", "No parameters given!"))
params["sleep_time"] = params.get("sleep_time", 0.2)

if not params:
    environment.log_error("No parameters given!")
    

DUFT Config: duft-config
DUFT Config: duft-config
[2024-08-27T17:10:29.699602] Jupyter Sample Data Task - INFO: Jupyter Sample Data Task initialised running in active mode
>>> Jupyter Sample Data Task UPDATE: Jupyter Sample Data Task initialised running in active mode


# Import dependancies

In [39]:
import jaydebeapi
import pandas as pd
from pandas import DataFrame
from sqlalchemy import create_engine
from jaydebeapi import Connection
import time
from datetime import datetime

# Set Source and Destinaton Database Connections 

In [40]:
source_environment = get_resolved_parameters_for_connection("EPMS_Source")
destination_environment = get_resolved_parameters_for_connection("EPMS_Destination")

DUFT Config: duft-config
DUFT Config: duft-config


# Set Filemaker Driver Files

In [41]:
driver_jar_path = "/Library/Java/Extensions/fmjdbc.jar"
driver_class = "com.filemaker.jdbc.Driver"

# Process File Maker Data

In [42]:
def open_file_maker_connection():
    try:
        server = source_environment.get("server")
        username = source_environment.get("username")
        password = source_environment.get("password")
        return jaydebeapi.connect(driver_class, server, [username, password], driver_jar_path)
    except Exception as ex:
        return None


def close_file_maker_connection(con: Connection):
    if con is not None:
        con.close()


def fetch_file_maker_data(query: str, con: Connection):
    cursor = con.cursor()
    cursor.execute(query)
    data_rows = cursor.fetchall()
    cursor.close()
    return data_rows

# Open connection to FM

In [43]:
fm_connection = open_file_maker_connection()

# Create dim_patient Data Frame

In [44]:
patient_confs = {
    "query": "SELECT  r.name, d.name, f.name, f.code, p.id, p.sex, p.dob, p.maritalStatus, p.patientStatus, p.clientCode, pharmacyNumber, pmtctNumber, nameFirst, nameLast, phoneCellNumber, p.ageYear, resCurrentAddTown, resCurrentAddConstituency, resCurrentAddStreet, resPermanentAddTown, resPermanentAddConstituency, resPermanentAddStreet, idFacilityCurrent,  homeBasedCareOrg, homeBasedCareCode,  deathDate  FROM pat p inner join fac f on p.idFacilityCurrent = f.id  inner join region r on f.idregion=r.id inner join district d on f.idDistrict = d.id",
    "cols": ['region','district','current_facility', 'facility_code','client_id', 'sex', 'date_of_birth', 'marital_status','patient_status','client_code', 'pharmacy_code','pmtct_number', 'first_name','last_name','contact_number','age','current_town','current_constituency','current_street', 'permanent_town','permanent_constituency','permanent_street','id_facility_current', 'cbart_cargs_name', 'cbart_cargs_code', 'death_date']
}

patient_query = patient_confs.get("query")
patient_cols = patient_confs.get("cols")

patient_data = fetch_file_maker_data(patient_query, fm_connection)
dim_pat_df = pd.DataFrame(patient_data, columns=patient_cols)

dim_pat_df['date_of_birth'] = pd.to_datetime(dim_pat_df['date_of_birth'], errors='coerce')
dim_pat_df['first_name'] = 'FName'
dim_pat_df['last_name'] = 'LName'
dim_pat_df['contact_number'] = '123456'

dim_pat_df.head()


,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,current_town,current_constituency,current_street,permanent_town,permanent_constituency,permanent_street,id_facility_current,cbart_cargs_name,cbart_cargs_code,death_date
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,None,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,None,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,None,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,None,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,2007-12-31
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,WINDHOEK,Katutura Central,ERF 54 WANAHEDA,WINDHOEK,Katutura Central,ERF 54 WANAHEDA,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,OMWENE TUUDA,None,None


# Get patient latest transfer status

In [45]:
fact_tsfr_confs = {
    "query": "SELECT idpatient,  status, \"date\" FROM tsfr",
    "cols": ['client_id' ,'transfer_status', 'transfer_date']
}

fact_tsfr_query = fact_tsfr_confs.get("query")
fact_tsfr_cols = fact_tsfr_confs.get("cols")

fact_tsfr_data = fetch_file_maker_data(fact_tsfr_query, fm_connection)
fact_tsfr_df = pd.DataFrame(fact_tsfr_data, columns=fact_tsfr_cols)

fact_tsfr_df = fact_tsfr_df.dropna(subset=['transfer_date'])

fact_tsfr_df = fact_tsfr_df.loc[fact_tsfr_df.groupby('client_id')['transfer_date'].idxmin()].reset_index(drop=True)
fact_tsfr_df.head()

fact_tsfr_df.head(100)

,client_id,transfer_status,transfer_date
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,Out,2018-01-05
1,0003CE1A-FDA7-0644-AC8A-6A2AC22F458C,Out,2009-06-01
2,000C0885-8782-1047-9575-2096412DF234,Out,2015-11-04
3,000FC632-25F4-F446-B53B-79332149CDE0,Out,2010-10-13
4,00148C55-3E2C-264B-86A9-213C01DBD15E,Out Pending,2024-01-22
...,...,...,...
95,02CB134D-12C7-9D43-A912-772007E0A5C2,In,2017-06-27
96,02CC9659-BC19-4B46-91C1-8D48653FC02E,In,2011-01-03
97,02CD8D38-5A84-124D-B51E-6E302BB7C5DE,Out,2007-05-15
98,02D07FE0-C4BE-924E-9936-F1F6F069AE54,In,2021-01-21


# Add transfer status to patient

In [46]:
dim_pat_df = pd.merge(dim_pat_df, fact_tsfr_df, on='client_id', how='left')

dim_pat_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,current_street,permanent_town,permanent_constituency,permanent_street,id_facility_current,cbart_cargs_name,cbart_cargs_code,death_date,transfer_status,transfer_date
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,NaN,NaN
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,NaN,NaN
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,NaN,NaN
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,2007-12-31,NaN,NaN
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,ERF 54 WANAHEDA,WINDHOEK,Katutura Central,ERF 54 WANAHEDA,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,OMWENE TUUDA,None,None,NaN,NaN


# Get first Weight and WHO Stage

In [47]:
fact_meas_confs = {
    "query": "SELECT idpatient, weight, whoStage,\"Date\" FROM Meas m",
    "cols": ['client_id', 'weight','who_stage', 'creation_date']
}

fact_meas_query = fact_meas_confs.get("query")
fact_meas_cols = fact_meas_confs.get("cols")

fact_meas_data = fetch_file_maker_data(fact_meas_query, fm_connection)
fact_meas_df = pd.DataFrame(fact_meas_data, columns=fact_meas_cols)

fact_meas_df = fact_meas_df.dropna(subset=['creation_date'])

fact_meas_df = fact_meas_df.loc[fact_meas_df.groupby('client_id')['creation_date'].idxmin()].reset_index(drop=True)
fact_meas_df.head()

fact_meas_df.head(100)

,client_id,weight,who_stage,creation_date
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,0.0,1,2014-03-18
1,000C0885-8782-1047-9575-2096412DF234,0.0,2,2011-06-16
2,00148C55-3E2C-264B-86A9-213C01DBD15E,0.0,1,2005-05-09
3,001946D5-919B-CB43-9B0E-68C03FF5D595,0.0,None,2010-06-24
4,002D86C5-77BF-184E-A9E4-AFBD55EEA37A,56.8,1,2018-07-30
...,...,...,...,...
95,02AB4B71-C650-D147-A789-D32ED3A6D813,58.2,1,2018-08-20
96,02B17343-92BE-024F-AEBE-4EEAEF6A2163,47.0,1,2019-06-11
97,02B30FFB-8642-0D41-BE3E-5F72B8C698F9,84.0,1,2018-10-10
98,02BADAAF-A923-D445-AC46-290BEDBE6610,0.0,2,2017-12-28


# Create fact_hiv_diagnosis Data Frame

In [48]:
fact_hiv_diagnosis_confs = {
    "query": "SELECT idpatient, hivconfirmationdate, hivconfirmatoryresultsdate, hivconfirmatoryresultstype, idfacilitycreate, fullDisclosureDAte FROM Cd",
    "cols": ['client_id', 'hiv_confirmation_date','hiv_confirmatory_result_date', 'hiv_confirmatory_result_type', 'hiv_diagnosis_facility_id','full_disclosure_date']
}

fact_hiv_diagnosis_query = fact_hiv_diagnosis_confs.get("query")
fact_hiv_diagnosis_cols = fact_hiv_diagnosis_confs.get("cols")

fact_hiv_diagnosis_data = fetch_file_maker_data(fact_hiv_diagnosis_query, fm_connection)
fact_hiv_diagnosis_df = pd.DataFrame(fact_hiv_diagnosis_data, columns=fact_hiv_diagnosis_cols)

fact_hiv_diagnosis_df['hiv_confirmation_date'] = pd.to_datetime(fact_hiv_diagnosis_df['hiv_confirmation_date'], errors='coerce')
fact_hiv_diagnosis_df['hiv_confirmatory_result_date'] = pd.to_datetime(fact_hiv_diagnosis_df['hiv_confirmatory_result_date'], errors='coerce')

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.dropna(subset=['hiv_confirmation_date'])

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.loc[fact_hiv_diagnosis_df.groupby('client_id')['hiv_confirmation_date'].idxmin()].reset_index(drop=True)

fact_hiv_diagnosis_df.head()

,client_id,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,full_disclosure_date
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,2014-03-18,NaT,None,DD2B5307-E5F8-484C-9E48-611A6BB41C1B,None
1,0003CE1A-FDA7-0644-AC8A-6A2AC22F458C,2005-07-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
2,00056F4E-16F0-6A4B-87BD-B3194199BB72,2005-01-15,NaT,None,B8A72A91-AF0F-4B25-A530-0CCF566FC0E3,None
3,0006BF9C-D80B-2545-9978-AAF9E56604EC,2009-08-19,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,None
4,000C0885-8782-1047-9575-2096412DF234,2011-05-18,NaT,None,7C92EA36-6301-46DB-8F2E-EC09CA1F1454,None


# Add Initial weight and WHO Stage to fact_hiv_diagnosis

In [49]:
fact_hiv_diagnosis_df = pd.merge(fact_hiv_diagnosis_df, fact_meas_df[['client_id','weight', 'who_stage']], on='client_id', how='left')

fact_hiv_diagnosis_df.head()

,client_id,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,full_disclosure_date,weight,who_stage
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,2014-03-18,NaT,None,DD2B5307-E5F8-484C-9E48-611A6BB41C1B,None,0.0,1
1,0003CE1A-FDA7-0644-AC8A-6A2AC22F458C,2005-07-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
2,00056F4E-16F0-6A4B-87BD-B3194199BB72,2005-01-15,NaT,None,B8A72A91-AF0F-4B25-A530-0CCF566FC0E3,None,NaN,NaN
3,0006BF9C-D80B-2545-9978-AAF9E56604EC,2009-08-19,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,None,NaN,NaN
4,000C0885-8782-1047-9575-2096412DF234,2011-05-18,NaT,None,7C92EA36-6301-46DB-8F2E-EC09CA1F1454,None,0.0,2


# Add dim_patient, fact_hiv_diagnosis to fact_sentinel_event dataframe

In [50]:
fact_sentinel_event_df = pd.merge(dim_pat_df, fact_hiv_diagnosis_df, on='client_id', how='left')

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,death_date,transfer_status,transfer_date,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,full_disclosure_date,weight,who_stage
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,NaN,NaN,1996-09-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,NaN,NaN,2001-09-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,2007-12-31,NaN,NaN,1997-10-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,None,NaN,NaN,1999-03-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,70.2,1


# Create hiv_enrolment dataframe 

In [51]:
fact_hiv_enrolment_confs = {
    "query": "SELECT c.idpatient, c.idfacilitycreate, c.hivenrolleddate, c.disclosureenrollmentdate, c.arteligiblereason, c.artstartdate, p.artnumber, p.artnumber, p.artnumberlegacy, c.idfacilityARTStart, tbScreenResult FROM pat p LEFT JOIN Cd c ON p.id=c.idpatient LEFT JOIN ia a  ON p.id=a.idpatient",
    "cols": ['client_id', 'hiv_enrollment_facility_id', 'hiv_enrollment_date', 'hiv_disclosure_enrollment_date','art_eligible_reason','art_start_date', 'quantum_number' , 'art_number', 'art_number_legacy', 'arv_initiating_facility','tb_screen_result']
}


fact_hiv_enrolment_query = fact_hiv_enrolment_confs.get("query")
fact_hiv_enrolment_cols = fact_hiv_enrolment_confs.get("cols")

fact_hiv_enrolment_data = fetch_file_maker_data(fact_hiv_enrolment_query, fm_connection)
fact_hiv_enrolment_df = pd.DataFrame(fact_hiv_enrolment_data, columns=fact_hiv_enrolment_cols)

fact_hiv_enrolment_df['hiv_enrollment_date'] = pd.to_datetime(fact_hiv_enrolment_df['hiv_enrollment_date'], errors='coerce')
fact_hiv_enrolment_df['hiv_disclosure_enrollment_date'] = pd.to_datetime(fact_hiv_enrolment_df['hiv_disclosure_enrollment_date'], errors='coerce')
fact_hiv_enrolment_df['art_start_date'] = pd.to_datetime(fact_hiv_enrolment_df['art_start_date'], errors='coerce')

fact_hiv_enrolment_df = fact_hiv_enrolment_df.dropna(subset=['hiv_enrollment_date'])

fact_hiv_enrolment_df = fact_hiv_enrolment_df.loc[fact_hiv_enrolment_df.groupby('client_id')['hiv_enrollment_date'].idxmin()].reset_index(drop=True)

fact_hiv_enrolment_df.head()

,client_id,hiv_enrollment_facility_id,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligible_reason,art_start_date,quantum_number,art_number,art_number_legacy,arv_initiating_facility,tb_screen_result
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,DD2B5307-E5F8-484C-9E48-611A6BB41C1B,2014-03-18,2016-12-06,CD4 only,2014-09-24,10476-0314-338,10476-0314-338,531031400052,DD2B5307-E5F8-484C-9E48-611A6BB41C1B,None
1,0003CE1A-FDA7-0644-AC8A-6A2AC22F458C,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2005-08-17,NaT,CD4 # or %,2006-08-15,10310-0805-76717,10310-0805-76717,115080504458,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
2,00056F4E-16F0-6A4B-87BD-B3194199BB72,B8A72A91-AF0F-4B25-A530-0CCF566FC0E3,2005-01-04,NaT,Clinical & CD4,NaT,12217-0105-91004,12217-0105-91004,134010500232,None,None
3,0006BF9C-D80B-2545-9978-AAF9E56604EC,1B258671-AF7E-4872-A207-1C10E742A8F6,2009-08-31,NaT,CD4 # or %,2013-07-12,10488-0809-87180,10488-0809-87180,127080915833,1B258671-AF7E-4872-A207-1C10E742A8F6,None
4,000C0885-8782-1047-9575-2096412DF234,7C92EA36-6301-46DB-8F2E-EC09CA1F1454,2011-06-16,NaT,Clinical & CD4,2014-08-07,10195-0611-74239,10195-0611-74239,105061101948,7C92EA36-6301-46DB-8F2E-EC09CA1F1454,None


# Add fact_hiv_enrolment to fact_sentinel_event dataframe

In [52]:
fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_hiv_enrolment_df, on='client_id', how='left')
fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,hiv_enrollment_facility_id,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligible_reason,art_start_date,quantum_number,art_number,art_number_legacy,arv_initiating_facility,tb_screen_result
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-07-24,NaT,Clinical & CD4,2004-03-25,10310-0703-72239,10310-0703-72239,115070300026,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-09-02,NaT,Clinical only,2003-09-18,10310-0903-72240,10310-0903-72240,115090300076,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-28,NaT,CD4 # or %,2003-08-28,10310-0803-72241,10310-0803-72241,115080300051,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-12,NaT,CD4 # or %,2003-08-12,10310-0803-72242,10310-0803-72242,115080300052,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-07-08,NaT,Clinical & CD4,2003-08-22,10310-0703-72243,10310-0703-72243,115070300002,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,Incomplete


# Get first and last visit ids per patient

In [53]:
fact_visits_confs = {
    "query": "SELECT id, idpatient, visitDate FROM Fup",
    "cols": ['id', 'client_id', 'visit_date' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

fact_visits_data = fetch_file_maker_data(fact_visits_query, fm_connection)
fact_visits_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

latest_visits_df = fact_visits_df.loc[fact_visits_df.groupby('client_id')['visit_date'].idxmax()]

first_visits_df = fact_visits_df.loc[fact_visits_df.groupby('client_id')['visit_date'].idxmin()]

# Create first_fact_visits dataframe 

In [54]:
fact_visits_confs = {
    "query": "SELECT DISTINCT f.idpatient, f.idfacilitycreate, f.visitDate FROM Fup f",
    "cols": ['client_id', 'visit_facility_id', 'visit_date']
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

ids = first_visits_df['id'].tolist()

ids_str = ','.join(f"'{id_}'" for id_ in ids)

fact_visits_query_updated = f"{fact_visits_query} WHERE f.id IN ({ids_str})"

fact_visits_data = fetch_file_maker_data(fact_visits_query_updated, fm_connection)
fact_first_visit_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

fact_first_visit_df['visit_date'] = pd.to_datetime(fact_first_visit_df['visit_date'], errors='coerce')

fact_first_visit_df.head() 

,client_id,visit_facility_id,visit_date
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,DD2B5307-E5F8-484C-9E48-611A6BB41C1B,2014-03-26
1,0003CE1A-FDA7-0644-AC8A-6A2AC22F458C,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2005-08-17
2,0006BF9C-D80B-2545-9978-AAF9E56604EC,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2013-07-12
3,000C0885-8782-1047-9575-2096412DF234,7C92EA36-6301-46DB-8F2E-EC09CA1F1454,2011-06-16
4,000FC632-25F4-F446-B53B-79332149CDE0,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2009-11-13


# Add first visit info to fact_sentinel_event

In [55]:
columns_to_add = {
    'visit_date': 'first_visit_date', 
    'visit_facility_id': 'first_visit_facility_id'
}

fact_first_visit_df = fact_first_visit_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_visit_df, on='client_id', how='left')
fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,hiv_disclosure_enrollment_date,art_eligible_reason,art_start_date,quantum_number,art_number,art_number_legacy,arv_initiating_facility,tb_screen_result,first_visit_date,first_visit_facility_id
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,NaT,Clinical & CD4,2004-03-25,10310-0703-72239,10310-0703-72239,115070300026,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,2004-03-25,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,NaT,Clinical only,2003-09-18,10310-0903-72240,10310-0903-72240,115090300076,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,2003-09-02,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,NaT,CD4 # or %,2003-08-28,10310-0803-72241,10310-0803-72241,115080300051,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,2003-08-28,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,NaT,CD4 # or %,2003-08-12,10310-0803-72242,10310-0803-72242,115080300052,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,2003-08-12,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,NaT,Clinical & CD4,2003-08-22,10310-0703-72243,10310-0703-72243,115070300002,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,Incomplete,2003-08-22,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E


# Create fact_last_visit dataframe 

In [56]:
fact_visits_confs = {
    "query": "SELECT DISTINCT f.idpatient, f.idfacilitycreate, f.visitDate, f.followupDate, f.scheduledDate, f.careModel, f.pregnantstatus, f.Breastfeeding, f.pregnantlmp, f.pregnantedd, f.ctxadherence, f.arvadherence, f.caremodel, r.code, r.line, cc.treatment, cc.dateTreatment, labResult, pf.stiGenitalUlcers, pf.stiVaginalUrethralDischarge, pf.stiScreenResult, f.oiDetail, f.oiOther FROM Fup f LEFT JOIN patFup pf on f.id=pf.id LEFT JOIN cc ON f.id=cc.idFollowUp LEFT JOIN rgm r ON f.id=r.idFollowUp",
    "cols": ['client_id', 'visit_facility_id', 'visit_date', 'next_visit_date', 'scheduled_visit_date', 'care_model', 'pregnancy_status', 'breast_feeding', 'lmp', 'edd', 'ctx_adherence', 'arv_adherence','care_model','current_regimen_code','current_regimen_line', 'cc_treatment_type','cc_treatment_date','cc_results', 'genital_ulcers','vaginal_urethral_discharge','sti_screening_result', 'oi', 'oi_other' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

ids = latest_visits_df['id'].tolist()

ids_str = ','.join(f"'{id_}'" for id_ in ids)

fact_visits_query_updated = f"{fact_visits_query} WHERE f.id IN ({ids_str})"

fact_visits_data = fetch_file_maker_data(fact_visits_query_updated, fm_connection)
fact_last_visit_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

fact_last_visit_df['visit_date'] = pd.to_datetime(fact_last_visit_df['visit_date'], errors='coerce')
fact_last_visit_df['next_visit_date'] = pd.to_datetime(fact_last_visit_df['next_visit_date'], errors='coerce')

fact_last_visit_df.head() 

,client_id,visit_facility_id,visit_date,next_visit_date,scheduled_visit_date,care_model,pregnancy_status,breast_feeding,lmp,edd,...,current_regimen_code,current_regimen_line,cc_treatment_type,cc_treatment_date,cc_results,genital_ulcers,vaginal_urethral_discharge,sti_screening_result,oi,oi_other
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2023-06-14,2023-11-29,2023-06-07,ART Clinic,No,No,None,None,...,None,None,None,None,None,No,No,Completed,None,None
1,0003CE1A-FDA7-0644-AC8A-6A2AC22F458C,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2009-05-19,2009-08-11,2009-05-19,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,0006BF9C-D80B-2545-9978-AAF9E56604EC,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2014-01-29,2014-02-26,2014-01-29,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,000C0885-8782-1047-9575-2096412DF234,58F993FA-A0CC-4AFE-8DA2-8B9EFD60E898,2020-07-13,2020-11-13,2019-06-07,None,None,None,None,None,...,ABC/3TC/EFV (99 / 00),2,None,None,None,None,None,None,None,None
4,000FC632-25F4-F446-B53B-79332149CDE0,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2010-08-24,2010-10-19,2010-08-24,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add last visit info to fact_sentinel_event

In [57]:
columns_to_add = {
    'visit_date': 'last_visit_date',
    'next_visit_date': 'last_next_visit_date', 
    'scheduled_visit_date': 'last_scheduled_visit_date', 
    'pregnancy_status': 'last_pregnancy_status', 
    'breast_feeding': 'last_breast_feeding', 
    'lmp': 'last_lmp', 
    'edd': 'last_edd',
    'visit_facility_id': 'last_visit_facility_id',
    'cc_treatment_type':'last_cc_treatment_type',
    'cc_treatment_date': 'last_cc_treatment_date',
    'cc_results': 'last_cc_results',
    'oi':'last_oi', 
    'oi_other': 'last_oi_other',
    'current_regimen_line': 'last_regimen_line'
}

fact_last_visit_df = fact_last_visit_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

max_last_visit_date = fact_last_visit_df['last_visit_date'].max()
print(f'last_refresh_date: {max_last_visit_date}')

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_visit_df, on='client_id', how='left')

fact_sentinel_event_df['last_visit_duration'] = (fact_sentinel_event_df['last_next_visit_date'].dt.year - fact_sentinel_event_df['last_visit_date'].dt.year) * 12 + (fact_sentinel_event_df['last_next_visit_date'].dt.month - fact_sentinel_event_df['last_visit_date'].dt.month)

fact_sentinel_event_df['iit_date'] = fact_sentinel_event_df['last_next_visit_date'] + pd.Timedelta(days=29)

fact_sentinel_event_df['iit_duration'] = (pd.Timestamp('today').normalize() - fact_sentinel_event_df['iit_date']).dt.days

fact_sentinel_event_df.head()


last_refresh_date: 2024-07-10 00:00:00


,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,last_visit_facility_id,last_cc_treatment_type,last_cc_treatment_date,last_cc_results,last_oi,last_oi_other,last_regimen_line,last_visit_duration,iit_date,iit_duration
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,None,None,None,0.0,2018-01-24,2407.0
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,None,None,None,3.0,2009-01-09,5709.0
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,Other,general body weakness,None,1.0,2003-10-10,7627.0
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,None,None,None,3.0,2007-11-06,6139.0
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,None,None,1A,6.0,2024-11-15,-80.0


# Create fact_tbt dataframe 

In [58]:
fact_tbt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, category, startDate, duration, stopDateExpected, stopDateActual , registrationNumber , site, siteDetail , outcome FROM tbt",
    "cols": ['client_id', 'tbt_facility_id', 'tbt_category', 'tbt_start_date', 'tbt_duration' ,'tbt_expected_stop_date', 'tbt_actual_stop_date', 'tbt_registration_number', 'tbt_site', 'tbt_site_detail', 'tbt_outcome']
}

fact_tbt_query = fact_tbt_confs.get("query")
fact_tbt_cols = fact_tbt_confs.get("cols")

fact_tbt_data = fetch_file_maker_data(fact_tbt_query, fm_connection)
fact_tbt_df = pd.DataFrame(fact_tbt_data, columns=fact_tbt_cols)

fact_tbt_df = fact_tbt_df.dropna(subset=['tbt_start_date'])

fact_tbt_df = fact_tbt_df.loc[fact_tbt_df.groupby('client_id')['tbt_start_date'].idxmax()]

fact_tbt_df.head()

,client_id,tbt_facility_id,tbt_category,tbt_start_date,tbt_duration,tbt_expected_stop_date,tbt_actual_stop_date,tbt_registration_number,tbt_site,tbt_site_detail,tbt_outcome
960,0031A23C-555E-1D41-BBDF-B28FB95252F7,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,New,2013-11-08,6.0,2014-05-08,2014-07-03,None,None,None,None
845,003BCB00-ECEA-E54C-B76B-7F3F5F3963E0,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,New,2016-10-18,6.0,2017-04-18,2017-04-16,None,None,None,None
1655,006FFF74-9567-B14E-8FF6-CBA65DEF423C,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,2020-01-29,6.0,2020-07-29,2021-01-06,None,PTB,None,None
1186,00B0A06B-0204-C54E-81B4-E3A496447896,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,New,2016-11-24,6.0,2017-05-24,None,None,None,None,None
342,00B34949-CA39-6E47-AD92-BE239C10480F,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,New,2006-01-09,6.0,2006-07-09,2006-09-18,None,None,None,None


# Add tbt info to fact_sentinel_event

In [59]:
columns_to_add = ['client_id', 'tbt_facility_id', 'tbt_category', 'tbt_start_date', 'tbt_duration' ,'tbt_expected_stop_date', 'tbt_actual_stop_date', 'tbt_registration_number', 'tbt_site', 'tbt_site_detail', 'tbt_outcome']

fact_tbt_df = fact_tbt_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_tbt_df, on='client_id', how='left')
fact_sentinel_event_df.head()

filtered_df = fact_sentinel_event_df[fact_sentinel_event_df['client_id'] == 'D2F9D495-4A71-B74B-B8C4-A04FA8369060']
filtered_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tbt_facility_id,tbt_category,tbt_start_date,tbt_duration,tbt_expected_stop_date,tbt_actual_stop_date,tbt_registration_number,tbt_site,tbt_site_detail,tbt_outcome
18702,Khomas,Windhoek,Okuryangava Clinic,10318,D2F9D495-4A71-B74B-B8C4-A04FA8369060,Female,1972-01-29,Widower,Lost,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create fact_tpt dataframe 

In [60]:
fact_tpt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, iptRegimen, startDate, duration, stopDateExpected, stopDateActual , CASE  WHEN stopReason = '1' THEN 'Completed TPT' WHEN stopReason = '2' THEN 'Cough' WHEN stopReason = '3' THEN 'Confirmed TB' WHEN stopReason = '4' THEN 'Hepatitis' WHEN stopReason = '5' THEN 'Neuropathy' WHEN stopReason = '6' THEN 'Poor Adherence' WHEN stopReason = '7' THEN 'Medicine Out Of Stock' WHEN stopReason = '8' THEN stopReasonOther ELSE '' END AS stopReason, status, adherenceDetail FROM ipt",
    "cols": ['client_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration' ,'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_stop_reason', 'tpt_status', 'tpt_adherance']
}

fact_tpt_query = fact_tpt_confs.get("query")
fact_tpt_cols = fact_tpt_confs.get("cols")

fact_tpt_data = fetch_file_maker_data(fact_tpt_query, fm_connection)
fact_tpt_df = pd.DataFrame(fact_tpt_data, columns=fact_tpt_cols)

fact_tpt_df['tpt_start_date'] = pd.to_datetime(fact_tpt_df['tpt_start_date'], errors='coerce')
fact_tpt_df['tpt_expected_stop_date'] = pd.to_datetime(fact_tpt_df['tpt_expected_stop_date'], errors='coerce')
fact_tpt_df['tpt_actual_stop_date'] = pd.to_datetime(fact_tpt_df['tpt_actual_stop_date'], errors='coerce')

fact_tpt_df['tpt_duration_two'] = (
    (fact_tpt_df['tpt_actual_stop_date'].dt.year - fact_tpt_df['tpt_start_date'].dt.year) * 12 +
    (fact_tpt_df['tpt_actual_stop_date'].dt.month - fact_tpt_df['tpt_start_date'].dt.month)
)

fact_tpt_df = fact_tpt_df.dropna(subset=['tpt_start_date'])

fact_tpt_df = fact_tpt_df.loc[fact_tpt_df.groupby('client_id')['tpt_start_date'].idxmax()]

fact_tpt_df.head()

,client_id,tpt_facility_id,tpt_regimen,tpt_start_date,tpt_duration,tpt_expected_stop_date,tpt_actual_stop_date,tpt_stop_reason,tpt_status,tpt_adherance,tpt_duration_two
9007,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,DD2B5307-E5F8-484C-9E48-611A6BB41C1B,None,2014-09-24,0.0,2014-09-24,2015-06-13,Completed TPT,Stopped,None,9.0
5248,000C0885-8782-1047-9575-2096412DF234,7C92EA36-6301-46DB-8F2E-EC09CA1F1454,None,2013-06-13,0.0,2013-06-13,2014-05-15,Completed TPT,Stopped,None,11.0
7975,00148C55-3E2C-264B-86A9-213C01DBD15E,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,3H,2022-03-03,0.0,2022-03-03,2022-03-03,Completed TPT,Stopped,None,0.0
7651,002D86C5-77BF-184E-A9E4-AFBD55EEA37A,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,6H,2017-11-16,6.0,2018-05-16,2018-04-30,Completed TPT,Stopped,None,5.0
10116,00327999-F20D-2743-85E0-E11277405059,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,2019-06-28,0.0,2019-06-28,2020-03-06,Completed TPT,Stopped,None,9.0


# Compute TPT status

In [61]:
def compute_tpt_status_two(row):
    if pd.notna(row['tpt_actual_stop_date']):
        if row['tpt_regimen'] == '3HP' and row['tpt_duration'] >= 70:
            return 'Up-to-date on TPT'
        elif row['tpt_regimen'] in ['9H','6H'] and row['tpt_duration'] >= 140:
            return 'Up-to-date on TPT'
        elif row['tpt_regimen'] in ['3H', '6H']:
            return 'Up-to-date on TPT'
    elif pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_actual_stop_date']):
        return 'Have not completed TPT'
    return 'Never initiated TPT'

fact_tpt_df['tpt_status_two'] = fact_tpt_df.apply(compute_tpt_status_two, axis=1)

fact_tpt_df.head()

,client_id,tpt_facility_id,tpt_regimen,tpt_start_date,tpt_duration,tpt_expected_stop_date,tpt_actual_stop_date,tpt_stop_reason,tpt_status,tpt_adherance,tpt_duration_two,tpt_status_two
9007,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,DD2B5307-E5F8-484C-9E48-611A6BB41C1B,None,2014-09-24,0.0,2014-09-24,2015-06-13,Completed TPT,Stopped,None,9.0,Never initiated TPT
5248,000C0885-8782-1047-9575-2096412DF234,7C92EA36-6301-46DB-8F2E-EC09CA1F1454,None,2013-06-13,0.0,2013-06-13,2014-05-15,Completed TPT,Stopped,None,11.0,Never initiated TPT
7975,00148C55-3E2C-264B-86A9-213C01DBD15E,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,3H,2022-03-03,0.0,2022-03-03,2022-03-03,Completed TPT,Stopped,None,0.0,Up-to-date on TPT
7651,002D86C5-77BF-184E-A9E4-AFBD55EEA37A,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,6H,2017-11-16,6.0,2018-05-16,2018-04-30,Completed TPT,Stopped,None,5.0,Up-to-date on TPT
10116,00327999-F20D-2743-85E0-E11277405059,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,2019-06-28,0.0,2019-06-28,2020-03-06,Completed TPT,Stopped,None,9.0,Never initiated TPT


# Add tpt info to fact_sentinel_event

In [62]:
columns_to_add = ['client_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration', 'tpt_duration_two' ,'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_stop_reason', 'tpt_status', 'tpt_status_two', 'tpt_adherance']

fact_tpt_df = fact_tpt_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_tpt_df, on='client_id', how='left')
fact_sentinel_event_df.head()

filtered_df = fact_sentinel_event_df[fact_sentinel_event_df['client_id'] == 'D2F9D495-4A71-B74B-B8C4-A04FA8369060']
filtered_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tpt_regimen,tpt_start_date,tpt_duration,tpt_duration_two,tpt_expected_stop_date,tpt_actual_stop_date,tpt_stop_reason,tpt_status,tpt_status_two,tpt_adherance
18702,Khomas,Windhoek,Okuryangava Clinic,10318,D2F9D495-4A71-B74B-B8C4-A04FA8369060,Female,1972-01-29,Widower,Lost,None,...,None,2018-03-23,0.0,13.0,2018-03-23,2019-04-04,Completed TPT,Stopped,Never initiated TPT,Good


# Create fact_lab dataframe 

In [63]:
fact_lab_confs = {
    "query": "SELECT l.idpatient, l.idfacilitycreate, l.orderDate, l.result, l.resultDate, lm.name, l.status FROM lab l INNER JOIN LabMaster lm ON lm.id=l.idtest",
    "cols": ['client_id', 'lab_facility_id', 'order_date', 'lab_result', 'result_date', 'test_name','status' ]
}

fact_lab_query = fact_lab_confs.get("query")
fact_lab_cols = fact_lab_confs.get("cols")

fact_lab_data = fetch_file_maker_data(fact_lab_query, fm_connection)
fact_lab_df = pd.DataFrame(fact_lab_data, columns=fact_lab_cols)

fact_lab_df['order_date'] = pd.to_datetime(fact_lab_df['order_date'], errors='coerce')
fact_lab_df['result_date'] = pd.to_datetime(fact_lab_df['result_date'], errors='coerce')


fact_lab_df.head()

,client_id,lab_facility_id,order_date,lab_result,result_date,test_name,status
0,ECF817DB-6F6F-4346-B06D-590F718C6FA4,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2022-03-16,196.0,2022-03-16,Viral Load,Complete
1,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-14,176.0,2003-08-14,CD4,Complete
2,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-14,12.6,2003-08-14,Hb,Complete
3,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-14,19.0,2003-08-14,ALT,Complete
4,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-09-02,63.0,2003-09-02,CD4,Complete


# Create fact_first_viral_load dataframe

In [64]:
fact_last_viral_load_tests_df = fact_lab_df[fact_lab_df['test_name'] == 'Viral Load']
fact_first_viral_load_tests_df = fact_last_viral_load_tests_df.sort_values('order_date').groupby('client_id').first().reset_index()

fact_first_viral_load_tests_df['lab_result'] = pd.to_numeric(fact_first_viral_load_tests_df['lab_result'], errors='coerce')

def categorize_result(result):
    if result < 40:
        return 'Suppressed'
    elif 40 <= result <= 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

fact_first_viral_load_tests_df['category'] = fact_first_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_first_viral_load_tests_df.head()

,client_id,lab_facility_id,order_date,lab_result,result_date,test_name,status,category
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,DD2B5307-E5F8-484C-9E48-611A6BB41C1B,2015-04-29,20.00,2015-04-29,Viral Load,Complete,Suppressed
1,000C0885-8782-1047-9575-2096412DF234,7C92EA36-6301-46DB-8F2E-EC09CA1F1454,2015-03-09,40.00,2015-03-09,Viral Load,Complete,LLV
2,000FC632-25F4-F446-B53B-79332149CDE0,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2010-04-13,260.39,2010-04-13,Viral Load,Complete,LLV
3,00148C55-3E2C-264B-86A9-213C01DBD15E,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2011-02-28,3091.38,2011-02-28,Viral Load,Complete,Unsuppressed
4,001946D5-919B-CB43-9B0E-68C03FF5D595,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2023-08-02,0.00,2023-08-02,Viral Load,Complete,Suppressed


# Create fact_last_viral_load dataframe

In [65]:
fact_last_viral_load_tests_df = fact_last_viral_load_tests_df.sort_values('order_date').groupby('client_id').last().reset_index()
fact_last_viral_load_tests_df['lab_result'] = pd.to_numeric(fact_last_viral_load_tests_df['lab_result'], errors='coerce')

def categorize_result(result):
    if result < 40:
        return 'Suppressed'
    elif 40 <= result <= 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

fact_last_viral_load_tests_df['category'] = fact_last_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_last_viral_load_tests_df.head()

,client_id,lab_facility_id,order_date,lab_result,result_date,test_name,status,category
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2023-05-10,0.00,2023-05-10,Viral Load,Complete,Suppressed
1,000C0885-8782-1047-9575-2096412DF234,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2020-01-22,0.00,2020-02-19,Viral Load,Complete,Suppressed
2,000FC632-25F4-F446-B53B-79332149CDE0,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2010-04-13,260.39,2010-04-13,Viral Load,Complete,LLV
3,00148C55-3E2C-264B-86A9-213C01DBD15E,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2023-11-01,40.00,2023-11-29,Viral Load,Complete,LLV
4,001946D5-919B-CB43-9B0E-68C03FF5D595,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2024-01-17,0.00,2024-01-17,Viral Load,Complete,Suppressed


# Add first viral load info to fact_sentinel_event

In [66]:
columns_to_add = {
    'order_date': 'first_viral_load_order_date',
    'result_date': 'first_viral_load_result_date', 
    'lab_result': 'first_viral_load_result',
    'lab_facility_id': 'first_viral_load_facility_id',
    'category': 'first_viral_load_result_category',
    'status': 'first_viral_load_result_status',
}

fact_first_viral_load_tests_df = fact_first_viral_load_tests_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_viral_load_tests_df, on='client_id', how='left')

fact_sentinel_event_df.head()

filtered_df = fact_sentinel_event_df[fact_sentinel_event_df['client_id'] == 'D2F9D495-4A71-B74B-B8C4-A04FA8369060']
filtered_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tpt_stop_reason,tpt_status,tpt_status_two,tpt_adherance,first_viral_load_order_date,first_viral_load_result_date,first_viral_load_result,first_viral_load_facility_id,first_viral_load_result_category,first_viral_load_result_status
18702,Khomas,Windhoek,Okuryangava Clinic,10318,D2F9D495-4A71-B74B-B8C4-A04FA8369060,Female,1972-01-29,Widower,Lost,None,...,Completed TPT,Stopped,Never initiated TPT,Good,2014-10-09,2014-10-09,80991.0,9A891DCC-149D-4B34-9A90-92A77852C802,Unsuppressed,Complete


# Add last viral load info to fact_sentinel_event

In [67]:
columns_to_add = {
    'order_date': 'last_viral_load_order_date',
    'result_date': 'last_viral_load_result_date', 
    'lab_result': 'last_viral_load_result',
    'lab_facility_id': 'last_viral_load_facility_id',
    'category': 'last_viral_load_result_category',
    'status': 'last_viral_load_result_status',
}

fact_last_visit_df = fact_last_viral_load_tests_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_visit_df, on='client_id', how='left')

# Add duration of ART in months

In [68]:
def diff_in_months(date1, date2):
    if pd.isnull(date1) or pd.isnull(date2):
        return None
    return (date1.year - date2.year) * 12 + date1.month - date2.month

fact_sentinel_event_df['months_since_art_start'] = fact_sentinel_event_df.apply(lambda row: diff_in_months(max_last_visit_date, row['art_start_date']), axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,first_viral_load_facility_id,first_viral_load_result_category,first_viral_load_result_status,last_viral_load_order_date,last_viral_load_result_date,last_viral_load_result,last_viral_load_facility_id,last_viral_load_result_category,last_viral_load_result_status,months_since_art_start
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,244.0
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,250.0
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,251.0
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,251.0
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,Unsuppressed,Complete,2023-03-17,2023-03-17,0.0,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,Suppressed,Complete,251.0


# Add program status

In [69]:
def classify_status(row):
    if pd.isna(row['death_date']) and row['patient_status'] != 'Transferred Out':
        if row['last_next_visit_date'] > max_last_visit_date:
            return 'Active on ART'
        elif (max_last_visit_date - row['last_next_visit_date']).days <= 28:
            return 'Active on ART: Missed Appointment'
        elif  29 < (max_last_visit_date - row['last_next_visit_date']).days <= 90:
            return 'IIT: Missed Appointment'
        elif pd.isnull(row['art_start_date']) and row['patient_status'] == 'Opt Out':
            return 'Never initiated on ART'
        elif (max_last_visit_date - row['last_next_visit_date']).days > 90:
            return 'Lost'
    elif pd.isna(row['death_date']):
        return 'Deceased'
    elif row['patient_status'] != 'Transferred Out':
        return 'Transferred Out'
    
fact_sentinel_event_df['client_status'] = fact_sentinel_event_df.apply(classify_status, axis=1)

fact_sentinel_event_df['tx_curr'] = fact_sentinel_event_df['client_status'].apply(
    lambda status: 'Yes' if status in ['Active on ART', 'Active on ART: Missed Appointment'] else 'No'
)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,first_viral_load_result_status,last_viral_load_order_date,last_viral_load_result_date,last_viral_load_result,last_viral_load_facility_id,last_viral_load_result_category,last_viral_load_result_status,months_since_art_start,client_status,tx_curr
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,244.0,Lost,No
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,250.0,Lost,No
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,251.0,Lost,No
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,251.0,Transferred Out,No
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,Complete,2023-03-17,2023-03-17,0.0,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,Suppressed,Complete,251.0,Active on ART,Yes


# Add patient ages at different stages cascade

In [70]:
fact_sentinel_event_df['hiv_confirmation_age'] = (fact_sentinel_event_df['hiv_confirmation_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['hiv_enrollment_age'] = (fact_sentinel_event_df['hiv_enrollment_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['first_viral_load_result_age'] = (fact_sentinel_event_df['first_viral_load_result_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['last_viral_load_result_age'] = (fact_sentinel_event_df['last_viral_load_result_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['first_visit_age'] = (fact_sentinel_event_df['first_visit_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['last_visit_age'] = (fact_sentinel_event_df['last_visit_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['current_age'] = (pd.to_datetime('today') - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,months_since_art_start,client_status,tx_curr,hiv_confirmation_age,hiv_enrollment_age,first_viral_load_result_age,last_viral_load_result_age,first_visit_age,last_visit_age,current_age
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,244.0,Lost,No,NaN,38.0,NaN,NaN,38.0,52.0,59.0
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,250.0,Lost,No,38.0,45.0,NaN,NaN,45.0,50.0,66.0
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,251.0,Lost,No,62.0,64.0,NaN,NaN,64.0,64.0,85.0
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,251.0,Transferred Out,No,22.0,28.0,NaN,NaN,28.0,32.0,49.0
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,251.0,Active on ART,Yes,37.0,42.0,46.0,61.0,42.0,62.0,63.0


# Add ART Duration

In [71]:
fact_sentinel_event_df['art_duration'] = (
    (max_last_visit_date.year - fact_sentinel_event_df['art_start_date'].dt.year) * 12 +
    (max_last_visit_date.month - fact_sentinel_event_df['art_start_date'].dt.month)
)

fact_sentinel_event_df['art_duration'] = (max_last_visit_date - fact_sentinel_event_df['art_start_date']).dt.days // 365.25

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,client_status,tx_curr,hiv_confirmation_age,hiv_enrollment_age,first_viral_load_result_age,last_viral_load_result_age,first_visit_age,last_visit_age,current_age,art_duration
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,Lost,No,NaN,38.0,NaN,NaN,38.0,52.0,59.0,20.0
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,Lost,No,38.0,45.0,NaN,NaN,45.0,50.0,66.0,20.0
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,Lost,No,62.0,64.0,NaN,NaN,64.0,64.0,85.0,20.0
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,Transferred Out,No,22.0,28.0,NaN,NaN,28.0,32.0,49.0,20.0
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,Active on ART,Yes,37.0,42.0,46.0,61.0,42.0,62.0,63.0,20.0


# Days since last visit

In [72]:
fact_sentinel_event_df['days_since_last_visit'] = (max_last_visit_date - fact_sentinel_event_df['last_visit_date']).dt.days
fact_sentinel_event_df['years_since_last_visit'] = (max_last_visit_date - fact_sentinel_event_df['last_visit_date']).dt.days // 365.25

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,hiv_confirmation_age,hiv_enrollment_age,first_viral_load_result_age,last_viral_load_result_age,first_visit_age,last_visit_age,current_age,art_duration,days_since_last_visit,years_since_last_visit
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,NaN,38.0,NaN,NaN,38.0,52.0,59.0,20.0,2402.0,6.0
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,38.0,45.0,NaN,NaN,45.0,50.0,66.0,20.0,5774.0,15.0
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,62.0,64.0,NaN,NaN,64.0,64.0,85.0,20.0,7622.0,20.0
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,22.0,28.0,NaN,NaN,28.0,32.0,49.0,20.0,6204.0,16.0
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,37.0,42.0,46.0,61.0,42.0,62.0,63.0,20.0,86.0,0.0


# Create dim age groups

In [73]:
bins = [0, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, float('inf')]
labels = ['0-4', '5-9', '10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64', '65+']

dim_age_group_df = pd.DataFrame({
    'current_age': range(1, 101)
})

dim_age_group_df['pepfar_age_group'] = pd.cut(dim_age_group_df['current_age'], bins=bins, labels=labels, right=False)

dim_age_group_df['paeds_adult_age_group'] = dim_age_group_df['current_age'].apply(lambda x: '0-18 yr old (Pediatric)' if x <= 18 else '19+ yr old (Adult)')

def tri_pillar_classify(age):
    if 0 <= age <= 18:
        return '0-18'
    elif age == 19:
        return '19'
    elif 20 <= age <= 39:
        return '20-39'
    else:
        return '39+'
    
dim_age_group_df['tri_pillar_age_group'] = dim_age_group_df['current_age'].apply(tri_pillar_classify)

dim_age_group_df.head(100)

,current_age,pepfar_age_group,paeds_adult_age_group,tri_pillar_age_group
0,1,0-4,0-18 yr old (Pediatric),0-18
1,2,0-4,0-18 yr old (Pediatric),0-18
2,3,0-4,0-18 yr old (Pediatric),0-18
3,4,5-9,0-18 yr old (Pediatric),0-18
4,5,5-9,0-18 yr old (Pediatric),0-18
...,...,...,...,...
95,96,65+,19+ yr old (Adult),39+
96,97,65+,19+ yr old (Adult),39+
97,98,65+,19+ yr old (Adult),39+
98,99,65+,19+ yr old (Adult),39+


# Add age group sort column

In [74]:
def tri_pillar_age_group_val(age):
    if 0 <= age <= 18:
        return 1
    elif age == 19:
        return 2
    elif 20 <= age <= 39:
        return 3
    else:
        return 4

dim_age_group_df['tri_pillar_age_group_val'] = dim_age_group_df['current_age'].apply(tri_pillar_age_group_val)

dim_age_group_df['paeds_adult_age_group_val'] = dim_age_group_df['paeds_adult_age_group'].apply(
    lambda x: 1 if x == '0-18 yr old (Pediatric)' else 2
)

def pepfar_age_group_val(label):
    if label == '0-4':
        return 1
    elif label == '5-9':
        return 2
    elif label == '10-14':
        return 3
    elif label == '15-19':
        return 4
    elif label == '20-24':
        return 5
    elif label == '25-29':
        return 6
    elif label == '30-34':
        return 7
    elif label == '35-39':
        return 8
    elif label == '40-44':
        return 9
    elif label == '45-49':
        return 10
    elif label == '50-54':
        return 11
    elif label == '55-59':
        return 12
    elif label == '60-64':
        return 13
    else:  # '65+'
        return 14

dim_age_group_df['pepfar_age_group_val'] = dim_age_group_df['pepfar_age_group'].apply(pepfar_age_group_val)

dim_age_group_df.head(100)

,current_age,pepfar_age_group,paeds_adult_age_group,tri_pillar_age_group,tri_pillar_age_group_val,paeds_adult_age_group_val,pepfar_age_group_val
0,1,0-4,0-18 yr old (Pediatric),0-18,1,1,1
1,2,0-4,0-18 yr old (Pediatric),0-18,1,1,1
2,3,0-4,0-18 yr old (Pediatric),0-18,1,1,1
3,4,5-9,0-18 yr old (Pediatric),0-18,1,1,2
4,5,5-9,0-18 yr old (Pediatric),0-18,1,1,2
...,...,...,...,...,...,...,...
95,96,65+,19+ yr old (Adult),39+,4,2,14
96,97,65+,19+ yr old (Adult),39+,4,2,14
97,98,65+,19+ yr old (Adult),39+,4,2,14
98,99,65+,19+ yr old (Adult),39+,4,2,14


# Create a column that contains current age groups

In [75]:
fact_sentinel_event_df = fact_sentinel_event_df.merge(
    dim_age_group_df[['current_age', 'pepfar_age_group', 'pepfar_age_group_val','paeds_adult_age_group', 'paeds_adult_age_group_val','tri_pillar_age_group','tri_pillar_age_group_val']], 
    on='current_age',
    how='left'
)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,current_age,art_duration,days_since_last_visit,years_since_last_visit,pepfar_age_group,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,59.0,20.0,2402.0,6.0,60-64,13,19+ yr old (Adult),2.0,39+,4.0
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,66.0,20.0,5774.0,15.0,65+,14,19+ yr old (Adult),2.0,39+,4.0
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,85.0,20.0,7622.0,20.0,65+,14,19+ yr old (Adult),2.0,39+,4.0
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,49.0,20.0,6204.0,16.0,50-54,11,19+ yr old (Adult),2.0,39+,4.0
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,63.0,20.0,86.0,0.0,60-64,13,19+ yr old (Adult),2.0,39+,4.0


# Calculate the difference in months between max_last_visit_date and last_viral_load_result_date

In [76]:
def calculate_months_since_last_viral_load(date):
    return (max_last_visit_date.year - date.year) * 12 + max_last_visit_date.month - date.month

fact_sentinel_event_df['months_since_last_viral_load'] = fact_sentinel_event_df['last_viral_load_result_date'].apply(calculate_months_since_last_viral_load)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,art_duration,days_since_last_visit,years_since_last_visit,pepfar_age_group,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,20.0,2402.0,6.0,60-64,13,19+ yr old (Adult),2.0,39+,4.0,NaN
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,20.0,5774.0,15.0,65+,14,19+ yr old (Adult),2.0,39+,4.0,NaN
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,20.0,7622.0,20.0,65+,14,19+ yr old (Adult),2.0,39+,4.0,NaN
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,20.0,6204.0,16.0,50-54,11,19+ yr old (Adult),2.0,39+,4.0,NaN
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,20.0,86.0,0.0,60-64,13,19+ yr old (Adult),2.0,39+,4.0,16.0


# Add VL Eligibility

In [77]:
def classify_vl_eligibility(row):
    if row['last_viral_load_result'] >= 40 and 3 < row['months_since_last_viral_load'] <= 6:
        return 'Recent (Slightly delayed)'
    elif (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and 3 < row['months_since_last_viral_load'] <= 6:
        return 'Recent (Slightly delayed)'
    elif row['current_age'] > 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and 12 < row['months_since_last_viral_load'] <= 18:
        return 'Recent (Slightly delayed)'
    elif row['current_age'] <= 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and 6 < row['months_since_last_viral_load'] <= 12:
        return 'Recent (Slightly delayed)'
    
    elif row['art_start_date'] <= max_last_visit_date - pd.DateOffset(days=180):
        if row['last_viral_load_result'] >= 40 and row['months_since_last_viral_load'] <= 3:
            return 'Monitored per Guidelines'
        elif (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['months_since_last_viral_load'] <= 3:
            return 'Monitored per Guidelines'
        elif row['current_age'] <= 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] <= 6:
            return 'Monitored per Guidelines'
        elif row['current_age'] > 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] <= 12:
            return 'Monitored per Guidelines'
    
    elif row['last_viral_load_result'] >= 40 and row['months_since_last_viral_load'] > 6:
        return 'Delayed'
    elif (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['months_since_last_viral_load'] > 6:
        return 'Delayed'
    elif row['current_age'] > 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] > 18:
        return 'Delayed'
    elif row['current_age'] <= 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] > 12:
        return 'Delayed'
    
    return 'Unclassified'  

fact_sentinel_event_df['vl_eligibility'] = fact_sentinel_event_df.apply(classify_vl_eligibility, axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,days_since_last_visit,years_since_last_visit,pepfar_age_group,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,vl_eligibility
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,2402.0,6.0,60-64,13,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,5774.0,15.0,65+,14,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,7622.0,20.0,65+,14,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,6204.0,16.0,50-54,11,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,86.0,0.0,60-64,13,19+ yr old (Adult),2.0,39+,4.0,16.0,Recent (Slightly delayed)


# Add reason for next VL

In [78]:
def categorize_vl_reason(row):
    if row['months_since_art_start'] is not None and row['months_since_art_start'] > 6 and pd.notnull(row['art_start_date']) and pd.isnull(row['first_viral_load_result_date']):
        return 'First Test'
    elif row['months_since_last_viral_load'] > 3 and row['last_pregnancy_status'] == 'Yes':
        return 'Pregnant Women'
    elif row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] == 'Yes':
        return 'BF Women'
    elif row['months_since_last_viral_load'] > 3 and row['last_viral_load_result_category'] == 'LLV':
        return 'Low Level Viraemia'
    elif row['months_since_last_viral_load'] > 6 and row['last_visit_age'] <= 19:
        return 'Children And Adolescents Under 19'
    elif row['months_since_last_viral_load'] > 12 and row['last_visit_age'] > 19:
        return 'Treatment Monitoring For Adults'
    else:
        return None

fact_sentinel_event_df['reason_for_next_vl'] = fact_sentinel_event_df.apply(categorize_vl_reason, axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,years_since_last_visit,pepfar_age_group,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,vl_eligibility,reason_for_next_vl
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,6.0,60-64,13,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,15.0,65+,14,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,20.0,65+,14,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,16.0,50-54,11,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,0.0,60-64,13,19+ yr old (Adult),2.0,39+,4.0,16.0,Recent (Slightly delayed),Treatment Monitoring For Adults


# Add next VL date

In [79]:
def calculate_next_vl_date(row):
    if pd.isnull(row['last_viral_load_result_date']):
        return None
    elif row['reason_for_next_vl'] == 'First Test':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Pregnant Women':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=3)
    elif row['reason_for_next_vl'] == 'BF Women':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=3)
    elif row['reason_for_next_vl'] == 'Low Level Viraemia':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Children And Adolescents Under 19':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Treatment Monitoring For Adults':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=12)
    else:
        return None

fact_sentinel_event_df['next_vl_date'] = fact_sentinel_event_df.apply(calculate_next_vl_date, axis=1)

print(fact_sentinel_event_df)

       region  district                current_facility facility_code  \
0      Khomas  Windhoek  Katutura Intermediate Hospital         10310   
1      Khomas  Windhoek  Katutura Intermediate Hospital         10310   
2      Khomas  Windhoek  Katutura Intermediate Hospital         10310   
3      Khomas  Windhoek  Katutura Intermediate Hospital         10310   
4      Khomas  Windhoek  Katutura Intermediate Hospital         10310   
...       ...       ...                             ...           ...   
19129  Khomas  Windhoek  Katutura Intermediate Hospital         10310   
19130  Khomas  Windhoek  Katutura Intermediate Hospital         10310   
19131  Khomas  Windhoek  Katutura Intermediate Hospital         10310   
19132  Khomas  Windhoek  Katutura Intermediate Hospital         10310   
19133  Khomas  Windhoek  Katutura Intermediate Hospital         10310   

                                  client_id     sex date_of_birth  \
0      EDC01C7E-D62B-574A-8A82-08E0182DC6E6  Female   

# Compute if patient has ever been initiated on ART  

In [80]:
def determine_has_been_initiated_on_art(row):
    return 'Yes' if pd.notna(row['art_start_date']) else 'No'
    
fact_sentinel_event_df['has_ever_been_initiated_on_art'] = fact_sentinel_event_df.apply(lambda row: determine_has_been_initiated_on_art(row), axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,13,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,14,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,14,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,11,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,13,19+ yr old (Adult),2.0,39+,4.0,16.0,Recent (Slightly delayed),Treatment Monitoring For Adults,2024-03-17,Yes


# Check if last VL is valid

In [81]:
def is_last_viral_load_valid(row, df):
    df_row = df.iloc[row.name]
    if df_row['months_since_last_viral_load'] <= 12:
        return 'Yes'
    elif pd.notna(df_row['art_start_date']):
        return 'No'
    else:
        return None
    
fact_sentinel_event_df['is_last_viral_load_valid'] = fact_sentinel_event_df.apply(lambda row: is_last_viral_load_valid(row, fact_sentinel_event_df), axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,19+ yr old (Adult),2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,19+ yr old (Adult),2.0,39+,4.0,16.0,Recent (Slightly delayed),Treatment Monitoring For Adults,2024-03-17,Yes,No


# Check if patient is virally suppressed

In [82]:
def determine_is_virally_suppressed(row, df):
    row = df.iloc[row.name]
    if row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result_category'] == 'Suppressed' and row['last_next_visit_date'] >= max_last_visit_date:
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['is_virally_suppressed'] = fact_sentinel_event_df.apply(determine_is_virally_suppressed, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid,is_virally_suppressed
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,2.0,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,2.0,39+,4.0,16.0,Recent (Slightly delayed),Treatment Monitoring For Adults,2024-03-17,Yes,No,None


# Check if patient is not virally suppressed

In [83]:
def determine_is_not_virally_suppressed(row, df):
    row = df.iloc[row.name]
    if row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= max_last_visit_date:
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['is_not_virally_suppressed'] = fact_sentinel_event_df.apply(determine_is_not_virally_suppressed, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid,is_virally_suppressed,is_not_virally_suppressed
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None,None
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None,None
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None,None
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,39+,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None,None
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,39+,4.0,16.0,Recent (Slightly delayed),Treatment Monitoring For Adults,2024-03-17,Yes,No,None,None


# Compute needs VL test

In [84]:
def determine_needs_VL_test(row, df):
    row = df.iloc[row.name]
    if (row['months_since_art_start'] > 6 and pd.notna(row['first_viral_load_result_date']) and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Suppressed' and row['last_next_visit_date'] >= max_last_visit_date and row['months_since_last_viral_load'] > 12 and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= max_last_visit_date and row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['months_since_art_start'] > 3 and pd.notna(row['first_viral_load_result_date']) and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Suppressed' and row['last_next_visit_date'] >= max_last_visit_date and row['months_since_last_viral_load'] > 6 and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= max_last_visit_date and row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['needs_vl_test'] = fact_sentinel_event_df.apply(determine_needs_VL_test, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tri_pillar_age_group_val,months_since_last_viral_load,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid,is_virally_suppressed,is_not_virally_suppressed,needs_vl_test
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None,None,None
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None,None,None
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None,None,None
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,4.0,NaN,Unclassified,First Test,NaT,Yes,No,None,None,None
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,4.0,16.0,Recent (Slightly delayed),Treatment Monitoring For Adults,2024-03-17,Yes,No,None,None,Yes


# Add Six MMP Eligibility

In [85]:
def determine_six_mmp_eligibility(row, df):
    row = df.iloc[row.name]
    if (row['current_age'] > 2 and (pd.isnull(row['tbt_start_date']) or row['tbt_duration'] >= 6) and (row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result'] < 40) and row['months_since_art_start'] >= 6  and (row['last_regimen_line'] == '1' or row['last_regimen_line'] == '2') and pd.isnull(row['last_oi_other'])):
        return 'Yes'
    else:
        return None

bins = [0, 3, 6, float('inf')]
labels = ['< 3 MMP', '3-5 MMP', '6+ MMP']

fact_sentinel_event_df['mmp_status'] = pd.cut(fact_sentinel_event_df['last_visit_duration'], bins=bins, labels=labels, right=False)

max_last_visit_month = max_last_visit_date.month
max_last_visit_year = max_last_visit_date.year

fact_sentinel_event_df['last_visit_month'] = fact_sentinel_event_df['last_visit_date'].dt.month
fact_sentinel_event_df['last_visit_year'] = fact_sentinel_event_df['last_visit_date'].dt.year

fact_sentinel_event_df['mmp_status_current'] = fact_sentinel_event_df.apply(
    lambda row: row['mmp_status'] if (row['last_visit_month'] == max_last_visit_month and row['last_visit_year'] == max_last_visit_year) else None,
    axis=1
)

fact_sentinel_event_df['mmp_status_adult'] = fact_sentinel_event_df['paeds_adult_age_group'].apply(lambda x: 'Yes' if 'adult' in str(x).lower() else None)

fact_sentinel_event_df['mmp_status_paed'] = fact_sentinel_event_df['paeds_adult_age_group'].apply(lambda x: 'Yes' if 'pediatric' in str(x).lower() else None)

fact_sentinel_event_df['six_mmp_eligible'] = fact_sentinel_event_df.apply(determine_six_mmp_eligibility, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,is_virally_suppressed,is_not_virally_suppressed,needs_vl_test,mmp_status,last_visit_month,last_visit_year,mmp_status_current,mmp_status_adult,mmp_status_paed,six_mmp_eligible
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,None,None,< 3 MMP,12.0,2017.0,None,Yes,None,None
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,None,None,3-5 MMP,9.0,2008.0,None,Yes,None,None
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,None,None,< 3 MMP,8.0,2003.0,None,Yes,None,None
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,None,None,3-5 MMP,7.0,2007.0,None,Yes,None,None
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,None,None,Yes,6+ MMP,4.0,2024.0,None,Yes,None,None


# Add Six MMP Eligibility But Not Given

In [86]:
fact_sentinel_event_df['six_mmp_eligible_but_not_given'] = fact_sentinel_event_df.apply(
    lambda row: 'Yes' if row['six_mmp_eligible'] == 'Yes' and row['mmp_status'] != '6+ MMP' else 'No', 
    axis=1
)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,is_not_virally_suppressed,needs_vl_test,mmp_status,last_visit_month,last_visit_year,mmp_status_current,mmp_status_adult,mmp_status_paed,six_mmp_eligible,six_mmp_eligible_but_not_given
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,None,< 3 MMP,12.0,2017.0,None,Yes,None,None,No
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,None,3-5 MMP,9.0,2008.0,None,Yes,None,None,No
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,None,< 3 MMP,8.0,2003.0,None,Yes,None,None,No
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,None,3-5 MMP,7.0,2007.0,None,Yes,None,None,No
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,None,Yes,6+ MMP,4.0,2024.0,None,Yes,None,None,No


# Compute currrent cascade status

In [87]:
def determine_art_outcomes(row):
    if row['is_virally_suppressed'] == 'Yes':
        return 'VL Suppressed'
    elif row['is_not_virally_suppressed'] == 'Yes':
        return 'VL Not Suppressed'
    elif row['needs_vl_test'] == 'Yes':
        return 'Needs VL Test'
    elif row['months_since_art_start'] < 6:
        return 'Recently initiated'
    elif row['has_ever_been_initiated_on_art'] == 'No':
        return 'Not initiated on ART'
    elif row['years_since_last_visit'] > 2:
        return 'Case Closed'
    elif row['patient_status'] == 'Deceased':
        return 'Died'
    else:
        return 'Unknown'

fact_sentinel_event_df['art_outcomes'] = fact_sentinel_event_df.apply(determine_art_outcomes, axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,needs_vl_test,mmp_status,last_visit_month,last_visit_year,mmp_status_current,mmp_status_adult,mmp_status_paed,six_mmp_eligible,six_mmp_eligible_but_not_given,art_outcomes
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,< 3 MMP,12.0,2017.0,None,Yes,None,None,No,Case Closed
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,3-5 MMP,9.0,2008.0,None,Yes,None,None,No,Case Closed
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,< 3 MMP,8.0,2003.0,None,Yes,None,None,No,Case Closed
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,3-5 MMP,7.0,2007.0,None,Yes,None,None,No,Case Closed
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,Yes,6+ MMP,4.0,2024.0,None,Yes,None,None,No,Needs VL Test


# Change dates from datetime to date 

In [88]:
fact_sentinel_event_df = fact_sentinel_event_df.apply(lambda col: col.dt.date if col.dtype == 'datetime64[ns]' else col)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,needs_vl_test,mmp_status,last_visit_month,last_visit_year,mmp_status_current,mmp_status_adult,mmp_status_paed,six_mmp_eligible,six_mmp_eligible_but_not_given,art_outcomes
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,< 3 MMP,12.0,2017.0,None,Yes,None,None,No,Case Closed
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,3-5 MMP,9.0,2008.0,None,Yes,None,None,No,Case Closed
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,< 3 MMP,8.0,2003.0,None,Yes,None,None,No,Case Closed
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,3-5 MMP,7.0,2007.0,None,Yes,None,None,No,Case Closed
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,Yes,6+ MMP,4.0,2024.0,None,Yes,None,None,No,Needs VL Test


# Clean up column names

In [89]:
def rename_columns(col):
    if '_id' in col:
        return '__' + col
    else:
        col = col.replace('_', ' ').title()
        col = col.replace('Tpt', 'TPT')
        col = col.replace('Tbt', 'TBT')
        col = col.replace('Hiv', 'HIV')
        col = col.replace('Art', 'ART')
        col = col.replace('Iit', 'IIT')
        col = col.replace('Mmp', 'MMP')
        col = col.replace('Pmtct', 'PMTCT')
        col = col.replace('Vl', 'Viral Load')
        col = col.replace('Tx Curr', 'TX Curr')
        col = col.replace('Oi', 'OI')
        col = col.replace('Lmp', 'LMP')
        col = col.replace('Edd', 'EDD')
        col = col.replace('Who', 'WHO')
        col = col.replace('Tb', 'TB')
        return col

fact_sentinel_event_df.columns = [rename_columns(col) for col in fact_sentinel_event_df.columns]

fact_sentinel_event_df.head()

,Region,District,Current Facility,Facility Code,__client_id,Sex,Date Of Birth,Marital Status,Patient Status,Client Code,...,Needs Viral Load Test,MMP Status,Last Visit Month,Last Visit Year,MMP Status Current,MMP Status Adult,MMP Status Paed,Six MMP Eligible,Six MMP Eligible But Not Given,ART Outcomes
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,< 3 MMP,12.0,2017.0,None,Yes,None,None,No,Case Closed
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,3-5 MMP,9.0,2008.0,None,Yes,None,None,No,Case Closed
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,< 3 MMP,8.0,2003.0,None,Yes,None,None,No,Case Closed
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,3-5 MMP,7.0,2007.0,None,Yes,None,None,No,Case Closed
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,Yes,6+ MMP,4.0,2024.0,None,Yes,None,None,No,Needs VL Test


# Define Destination Database Connection

In [93]:
server = destination_environment.get("server")
database = destination_environment.get("database")
port = destination_environment.get("port")
username = destination_environment.get("username")
password = destination_environment.get("password")

connection_url = ('postgresql://{0}:{1}@{2}:{3}/{4}'
                          .format(username, password, server, port, database))

engine = create_engine(connection_url)

# Write dim age group to table

In [94]:
table_name = 'dim_age_group'

dim_age_group_df.to_sql(table_name, con=engine, if_exists='replace', index=False, schema='analysis')

dim_age_group_df.head()

,current_age,pepfar_age_group,paeds_adult_age_group,tri_pillar_age_group,tri_pillar_age_group_val,paeds_adult_age_group_val,pepfar_age_group_val
0,1,0-4,0-18 yr old (Pediatric),0-18,1,1,1
1,2,0-4,0-18 yr old (Pediatric),0-18,1,1,1
2,3,0-4,0-18 yr old (Pediatric),0-18,1,1,1
3,4,5-9,0-18 yr old (Pediatric),0-18,1,1,2
4,5,5-9,0-18 yr old (Pediatric),0-18,1,1,2


# Write sentinel event to table

In [95]:
table_name = 'fact_sentinel_event'

fact_sentinel_event_df.to_sql(table_name, con=engine, if_exists='replace', index=False, schema='analysis')

fact_sentinel_event_df.head()

,Region,District,Current Facility,Facility Code,__client_id,Sex,Date Of Birth,Marital Status,Patient Status,Client Code,...,Needs Viral Load Test,MMP Status,Last Visit Month,Last Visit Year,MMP Status Current,MMP Status Adult,MMP Status Paed,Six MMP Eligible,Six MMP Eligible But Not Given,ART Outcomes
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,< 3 MMP,12.0,2017.0,None,Yes,None,None,No,Case Closed
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,3-5 MMP,9.0,2008.0,None,Yes,None,None,No,Case Closed
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,< 3 MMP,8.0,2003.0,None,Yes,None,None,No,Case Closed
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,3-5 MMP,7.0,2007.0,None,Yes,None,None,No,Case Closed
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,Yes,6+ MMP,4.0,2024.0,None,Yes,None,None,No,Needs VL Test


# Drop dataframes

In [96]:
del dim_pat_df
del fact_hiv_diagnosis_df
del fact_hiv_enrolment_df
del fact_first_visit_df
del fact_last_visit_df
del fact_visits_df
del fact_lab_df
del fact_tpt_df
del fact_meas_df
del fact_first_viral_load_tests_df
del fact_last_viral_load_tests_df
del fact_tbt_df
del fact_tsfr_df

# Log end time

In [97]:
pd.Timestamp.today()

Timestamp('2024-08-27 17:33:41.782177')